In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()

2023-02-27 21:13:32.218354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 21:13:33.350275: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-27 21:13:33.476466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-27 21:13:33.476493: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Task 0. L2 Regularization Cost
"""
Function that calculates the cost of a neural network with L2 Regularization
"""
def l2_reg_cost(cost, lambtha, weights, L, m):
    """
    cost: cost of the network without L2 regularization
    lambtha: regularization parameter
    weights: dictionary of the weights and biases of the neural network
    L: number of layers in the neural network
    m: number of data points used
    Returns the cost of the network accounting for L2 regularization
    """
    Frobenius_Norm = []
    for key, value in weights.items():
        if 'W' in key:
            Frobenius_Norm.append(np.sum(value * value))
    L2_norm = np.sum(Frobenius_Norm)
    return cost + (lambtha / (2 * m)) * L2_norm

In [3]:
# 0-main
np.random.seed(0)

weights = {}
weights['W1'] = np.random.randn(256, 784)
weights['W2'] = np.random.randn(128, 256)
weights['W3'] = np.random.randn(10, 128)

cost = np.abs(np.random.randn(1))

print(cost)
cost = l2_reg_cost(cost, 0.1, weights, 3, 1000)
print(cost)

[0.41842822]
[12.11229237]


In [4]:
# Task 1. Gradient Descent with L2 Regularization
"""
Function that updates the weights and biases of a neural network using
gradient descent with L2 regularization
"""
def l2_reg_gradient_descent(Y, weights, cache, alpha, lambtha, L):
    """
    Y: one-hot numpy.ndarray of shape (classes, m) that contains the correct
    labels for the data
        classes: number of classes
        m: number of data points
    weights: dictionary of the weights and biases of the neural network
    cache: dictionary of the outputs of each layer of the neural network
    alpha: learning rate
    lambtha: L2 regularization parameter
    L: number of layers of the network
    The neural network uses tanh activations on each layer except the last
    The last layer uses a softmax activation
    The weights and biases of the network should be updated in place
    """
    m = Y.shape[1]

    for layer in range(L, 0, -1):
        A_layer = cache["A{}".format(layer)]
        A_layer_prev = cache["A{}".format(layer - 1)]

        if layer == L:
            dz_layer = (A_layer - Y)
        else:
            dz_layer = dA_layer_prev * (1 - np.square(A_layer))

        W_layer = weights["W{}".format(layer)]

        dA_layer_prev = np.matmul(W_layer.T, dz_layer)

        dW_layer = (np.matmul(dz_layer, A_layer_prev.T) / m) +\
        ((lambtha / m) * W_layer)
        db_layer = np.sum(dz_layer, axis=1, keepdims=True) / m

        weights["W{}".format(layer)] -= alpha * dW_layer
        weights["b{}".format(layer)] -= alpha * db_layer


In [5]:
# 1-main
def one_hot(Y, classes):
    """convert an array to a one-hot matrix"""
    m = Y.shape[0]
    one_hot = np.zeros((classes, m))
    one_hot[Y, np.arange(m)] = 1
    return one_hot

lib= np.load('../data/MNIST.npz')
X_train_3D = lib['X_train']
Y_train = lib['Y_train']
X_train = X_train_3D.reshape((X_train_3D.shape[0], -1)).T
Y_train_oh = one_hot(Y_train, 10)

np.random.seed(0)

weights = {}
weights['W1'] = np.random.randn(256, 784)
weights['b1'] = np.zeros((256, 1))
weights['W2'] = np.random.randn(128, 256)
weights['b2'] = np.zeros((128, 1))
weights['W3'] = np.random.randn(10, 128)
weights['b3'] = np.zeros((10, 1))

cache = {}
cache['A0'] = X_train
cache['A1'] = np.tanh(np.matmul(weights['W1'], cache['A0']) + weights['b1'])
cache['A2'] = np.tanh(np.matmul(weights['W2'], cache['A1']) + weights['b2'])
Z3 = np.matmul(weights['W3'], cache['A2']) + weights['b3']
cache['A3'] = np.exp(Z3) / np.sum(np.exp(Z3), axis=0)
print(weights['W1'])
l2_reg_gradient_descent(Y_train_oh, weights, cache, 0.1, 0.1, 3)
print(weights['W1'])

[[ 1.76405235  0.40015721  0.97873798 ...  0.52130375  0.61192719
  -1.34149673]
 [ 0.47689837  0.14844958  0.52904524 ...  0.0960042  -0.0451133
   0.07912172]
 [ 0.85053068 -0.83912419 -1.01177408 ... -0.07223876  0.31112445
  -1.07836109]
 ...
 [-0.60467085  0.54751161 -1.23317415 ...  0.82895532  1.44161136
   0.18972404]
 [-0.41044606  0.85719512  0.71789835 ... -0.73954771  0.5074628
   1.23022874]
 [ 0.43129249  0.60767018 -0.07749988 ... -0.26611561  2.52287972
   0.73131543]]
[[ 1.76405199  0.40015713  0.97873779 ...  0.52130364  0.61192707
  -1.34149646]
 [ 0.47689827  0.14844955  0.52904513 ...  0.09600419 -0.04511329
   0.07912171]
 [ 0.85053051 -0.83912402 -1.01177388 ... -0.07223874  0.31112438
  -1.07836088]
 ...
 [-0.60467073  0.5475115  -1.2331739  ...  0.82895516  1.44161107
   0.189724  ]
 [-0.41044598  0.85719495  0.71789821 ... -0.73954756  0.5074627
   1.2302285 ]
 [ 0.4312924   0.60767006 -0.07749987 ... -0.26611556  2.52287922
   0.73131529]]


In [6]:
# Task 2. L2 Regularization Cost
"""
Function that calculates the cost of a neural network with L2 regularization
"""
def l2_reg_cost(cost):
    """
    cost: tensor containing the cost of the network without L2 reg
    Return a tensor containing the cost of the network accounting for L2 reg
    """
    return (cost + tf.losses.get_regularization_losses())

In [7]:
# 2-main
def one_hot(Y, classes):
    """convert an array to a one-hot matrix"""
    m = Y.shape[0]
    oh = np.zeros((classes, m))
    oh[Y, np.arange(m)] = 1
    return oh

np.random.seed(4)
m = np.random.randint(1000, 2000)
c = 10
lib= np.load('../data/MNIST.npz')

X = lib['X_train'][:m].reshape((m, -1))
Y = one_hot(lib['Y_train'][:m], c).T

n0 = X.shape[1]
n1, n2 = np.random.randint(10, 1000, 2)

lam = np.random.uniform(0.01)
tf.set_random_seed(0)

x = tf.placeholder(tf.float32, (None, n0))
y = tf.placeholder(tf.float32, (None, c))

a1 = tf.layers.Dense(n1, activation=tf.nn.tanh, kernel_initializer=tf.keras.initializers.VarianceScaling(mode="fan_avg"), kernel_regularizer=tf.keras.regularizers.l2(lam))(x)
a2 = tf.layers.Dense(n2, activation=tf.nn.sigmoid, kernel_initializer=tf.keras.initializers.VarianceScaling(mode="fan_avg"), kernel_regularizer=tf.keras.regularizers.l2(lam))(a1)
y_pred = tf.layers.Dense(c, activation=None, kernel_initializer=tf.keras.initializers.VarianceScaling(mode="fan_avg"), kernel_regularizer=tf.keras.regularizers.l2(lam))(a2)

cost = tf.losses.softmax_cross_entropy(y, y_pred)

l2_cost = l2_reg_cost(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(l2_cost, feed_dict={x: X, y: Y}))


2023-02-27 21:13:49.665997: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-27 21:13:49.666806: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-27 21:13:49.667783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (BensLaptop): /proc/driver/nvidia/version does not exist
2023-02-27 21:13:49.672903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 21:13:49.709628: I tensorflow/compiler/mli

[56.608963  49.789898   5.9518623]


In [8]:
# Task 3. Create a Layer with L2 Regularization
"""
Function that creates a tensorflow layer that includes L2 regularization
"""
def l2_reg_create_layer(prev, n, activation, lambtha):
    """
    prev: tensor containing the output of the previous layer
    n: number of nodes the new layer should contain
    activation: activation function that should be used on the layer
    lambtha: the L2 regularization parameter
    Returns the output of the new layer
    """
    l2_regular_layer = tf.layers.Dense(n,
                                      activation=activation,
                                      kernel_initializer=tf.keras.initializers.VarianceScaling(mode="fan_avg"),
                                      kernel_regularizer=tf.keras.regularizers.l2(lambtha))
    return l2_regular_layer(prev)

In [9]:
# 3-main
def one_hot(Y, classes):
    """convert an array to a one-hot matrix"""
    m = Y.shape[0]
    one_hot = np.zeros((m, classes))
    one_hot[np.arange(m), Y] = 1
    return one_hot

lib= np.load('../data/MNIST.npz')
X_train_3D = lib['X_train']
Y_train = lib['Y_train']
X_train = X_train_3D.reshape((X_train_3D.shape[0], -1))
Y_train_oh = one_hot(Y_train, 10)

tf.set_random_seed(0)
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])
h1 = l2_reg_create_layer(x, 256, tf.nn.tanh, 0.1)
y_pred = l2_reg_create_layer(x, 10, None, 0.)
cost = tf.losses.softmax_cross_entropy(y, y_pred)
l2_cost = l2_reg_cost(cost)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(l2_cost, feed_dict={x: X_train, y: Y_train_oh}))

[56.63227   49.813206   5.9751678 41.08217    2.4343293]


In [10]:
# Task 4. Forward Propagation with Dropout
"""
Function that conducts forward propagation using Dropout
"""
def dropout_forward_prop(X, weights, L, keep_prob):
    """
    X: numpy.ndarray of shape (nx, m) containing the input data for the network
        nx: number of input features
        m: number of data points
    weights: dictionary of the weights and biases of the neural network
    L: number of layers in the network
    keep_prob: the probability that a node will be kept
    All layers except the last should use the tanh activation function
    Last layer will use the softmax activation function
    Returns a dictionary containing the outputs of each layer and the dropout
        mask used on each layer
    """
    cache = {}
    A = X
    cache["A{}".format(0)] = X
    for layer in range(1, L + 1):
        W = weights["W{}".format(layer)]
        b = weights["b{}".format(layer)]
        Z = np.matmul(W, A) + b
        if layer == L:
            A = np.exp(Z) / np.sum(np.exp(Z), axis=0, keepdims=True)
            cache["A{}".format(layer)] = A
        else:
            A = np.tanh(Z)
            dropout_mask = np.random.binomial(n=1, p=keep_prob, size=A.shape)
            A = A * dropout_mask / keep_prob
            cache["A{}".format(layer)] = A
            cache["D{}".format(layer)] = dropout_mask
    return cache



In [11]:
def one_hot(Y, classes):
    """convert an array to a one-hot matrix"""
    m = Y.shape[0]
    one_hot = np.zeros((classes, m))
    one_hot[Y, np.arange(m)] = 1
    return one_hot

lib= np.load('../data/MNIST.npz')
X_train_3D = lib['X_train']
Y_train = lib['Y_train']
X_train = X_train_3D.reshape((X_train_3D.shape[0], -1)).T
Y_train_oh = one_hot(Y_train, 10)

np.random.seed(0)

weights = {}
weights['W1'] = np.random.randn(256, 784)
weights['b1'] = np.zeros((256, 1))
weights['W2'] = np.random.randn(128, 256)
weights['b2'] = np.zeros((128, 1))
weights['W3'] = np.random.randn(10, 128)
weights['b3'] = np.zeros((10, 1))

cache = dropout_forward_prop(X_train, weights, 3, 0.8)
for k, v in sorted(cache.items()):
    print(k, v)

A0 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
A1 [[-1.24999999 -1.25       -1.24999945 ... -1.25       -1.25
  -1.25      ]
 [ 1.25        1.24999777  1.25       ...  0.37738875  1.24999717
  -1.24999889]
 [ 0.19383179 -0.80653094 -1.24950714 ...  1.24253535  1.08653948
  -1.20190135]
 ...
 [-1.25       -1.25        0.         ... -0.         -1.25
  -1.24999852]
 [-1.0858595  -1.25        0.         ...  1.24972487 -0.88878698
  -1.24999933]
 [ 1.25        1.24999648  0.2057473  ...  0.          1.23194191
  -1.24908257]]
A2 [[-1.25        0.          1.24985922 ... -1.25        0.
   1.24996854]
 [-0.         -0.         -0.         ... -1.24996232 -0.70684864
   1.25      ]
 [-1.25        0.          0.18486152 ... -1.24999999 -1.25
  -1.24999989]
 ...
 [ 1.2404131   1.25        1.25       ...  1.1670038   1.25
  -0.        ]
 [ 1.25        1.25       -1.24998041 ...  1.2

In [12]:
# Task 5. Gradient Descent with Dropout
"""
Function that updates the weights of a neural network with Dropout
regularization using gradient descent
"""
def dropout_gradient_descent(Y, weights, cache, alpha, keep_prob, L):
    """
    Y: one-hot numpy.ndarray of shape (classes, m) that contains data labels
        classes: number of classes
        m: number of data points
    weights: dictionary of the weights and biases of the neural network
    cache: dictionary of the outputs and dropout masks of each layer of the nn
    alpha: learning rate
    keep_prob: probability that a node will be kept
    L: number of layers of the network
    All layers except the last use the tanh activation function
    The last layer uses the softmax activation function
    The weights of the network should be updated in place
    """
    m = Y.shape[1]

    for layer in range(L, 0, -1):
        A = cache["A{}".format(layer)]
        A_prev = cache["A{}".format(layer - 1)]

        if layer != L:
            dZ = dA_prev * (1 - (A ** 2))
            dZ = (dZ * cache["D{}".format(layer)]) / keep_prob
        else:
            dZ = A - Y
        W = weights["W{}".format(layer)]
        dW = np.matmul(dZ, A_prev.T) / m
        db = np.sum(dZ, axis=1, keepdims=True) / m
        dA_prev = np.matmul(W.T, dZ)

        weights["W{}".format(layer)] -= alpha * dW
        weights["b{}".format(layer)] -= alpha * db


In [13]:
# 5-main
def one_hot(Y, classes):
    """convert an array to a one-hot matrix"""
    m = Y.shape[0]
    one_hot = np.zeros((classes, m))
    one_hot[Y, np.arange(m)] = 1
    return one_hot

lib= np.load('../data/MNIST.npz')
X_train_3D = lib['X_train']
Y_train = lib['Y_train']
X_train = X_train_3D.reshape((X_train_3D.shape[0], -1)).T
Y_train_oh = one_hot(Y_train, 10)

np.random.seed(0)

weights = {}
weights['W1'] = np.random.randn(256, 784)
weights['b1'] = np.zeros((256, 1))
weights['W2'] = np.random.randn(128, 256)
weights['b2'] = np.zeros((128, 1))
weights['W3'] = np.random.randn(10, 128)
weights['b3'] = np.zeros((10, 1))

cache = dropout_forward_prop(X_train, weights, 3, 0.8)
print(weights['W2'])
dropout_gradient_descent(Y_train_oh, weights, cache, 0.1, 0.8, 3)
print(weights['W2'])

[[-1.9282086  -0.71324613 -1.33191318 ... -2.14202626 -0.07737407
   0.99832167]
 [-0.0237149  -0.18364778  0.08337452 ... -0.06093055 -0.03924408
  -2.17625294]
 [-0.16181888  0.49237435 -0.47196279 ...  0.97504077  0.16272698
   0.56159916]
 ...
 [ 0.39842474 -0.09870005  1.32173992 ... -0.33210834  0.66215988
   0.87211421]
 [ 0.15767221  0.42236212  1.004765   ...  0.69883284  0.70857088
  -0.44427252]
 [ 2.68588811 -0.60351958 -1.0759598  ... -1.2437044   0.69462324
   1.00090403]]
[[-1.92044686 -0.71894673 -1.32811693 ... -2.14071955 -0.07158198
   0.98206832]
 [-0.03706116 -0.17088483  0.07798748 ... -0.07245569 -0.0491215
  -2.16245276]
 [-0.17198668  0.49842244 -0.47369328 ...  0.96880194  0.15497217
   0.5693131 ]
 ...
 [ 0.41997262 -0.11452751  1.32873227 ... -0.31312321  0.67162237
   0.85928296]
 [ 0.13702353  0.44237056  1.00139188 ...  0.68128208  0.69020934
  -0.43055442]
 [ 2.66514017 -0.59204122 -1.08943163 ... -1.26238074  0.69280683
   1.02353101]]


In [14]:
# Task 6. Create a Layer with Dropout
"""
Function that creates a layer of a neural network using dropout
"""
def dropout_create_layer(prev, n, activation, keep_prob):
    """
    prev: a tensor containing the output of the previous layer
    n: number of nodes the new layer should contain
    activation: the activation function that should be used on the layer
    keep_prob: the probability that a node will be kept
    Returns the output of the new layer
    """
    init = tf.keras.initializers.VarianceScaling(mode="fan_avg")
    dropout_reg = tf.layers.Dropout(rate=keep_prob)
    layer = tf.layers.dense(inputs=prev,
                            units=n,
                            activation=activation,
                            kernel_initializer=init,
                            kernel_regularizer=dropout_reg)
    return layer



In [15]:
# 6-main
tf.set_random_seed(0)
np.random.seed(0)
x = tf.placeholder(tf.float32, shape=[None, 784])
X = np.random.randint(0, 256, size=(10, 784))
a = dropout_create_layer(x, 256, tf.nn.tanh, 0.8)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a, feed_dict={x: X}))

[[ 1.        -1.        -0.9997158 ...  1.        -1.        -1.       ]
 [ 1.        -1.         1.        ...  1.        -1.        -1.       ]
 [ 1.        -1.         1.        ...  1.        -1.        -1.       ]
 ...
 [ 1.        -1.         1.        ... -0.9998197 -1.        -1.       ]
 [ 1.        -1.        -1.        ... -1.        -1.         1.       ]
 [ 1.        -1.         1.        ...  1.        -1.        -1.       ]]


/tmp/ipykernel_1870/1688624747.py:15: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  layer = tf.layers.dense(inputs=prev,


In [28]:
# Task 7. Early stopping
"""
Function that determines if gradient descent should stop early
"""
def early_stopping(cost, opt_cost, threshold, patience, count):
    """
    cost: the current validation cost of the neural network
    opt_cost: the lowest recorded validation cost of the network
    threshold: the threshold used for early stopping
    patience: the patience count used for early stopping
    count: count of how long the threshold has not been met
    Returns a boolean of whether the network should be stopped early
        followed by the updated count
    """
    cost_gap = opt_cost - cost

    if cost_gap > threshold:
        # Reset count because cost_gap is larger than threshold
        count = 0
    else:
        # Add to count and wait patiently
        count += 1

    if count < patience:
        # We don't want to stop because we haven't waited long enough
        return(False, count)
    else:
        # We waited long enough and should stop early
        return(True, count)




In [29]:
# 7-main
print(early_stopping(1.0, 1.9, 0.5, 15, 5))
print(early_stopping(1.1, 1.5, 0.5, 15, 2))
print(early_stopping(1.0, 1.5, 0.5, 15, 8))
print(early_stopping(1.0, 1.5, 0.5, 15, 14))

(False, 0)
(False, 3)
(False, 9)
(True, 15)
